Импорт модулей

In [34]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import gensim
import gensim.downloader
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
import re
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\krygl\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

Считываем данные

In [35]:
df = pd.read_csv("data/sample-data.csv")
df.head()

,id,description
0,1,Active classic boxers - There's a reason why o...
1,2,Active sport boxer briefs - Skinning up Glory ...
2,3,Active sport briefs - These superbreathable no...
3,4,"Alpine guide pants - Skin in, climb ice, switc..."
4,5,"Alpine wind jkt - On high ridges, steep ice an..."


# TF-IDF

Предстим массив текстов в виде векторов

In [36]:
tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(df['description'])

cosine_similarities_tfidf = cosine_similarity(tfidf_matrix, tfidf_matrix)

Напишем функцию для получения названия товара (из описания) по его id

In [37]:
def item(id):
    return df.loc[df['id'] == id]['description'].tolist()[0].split(' - ')[0]

Напишем функцию для рекомендации наиболее похожих товаров для id

In [38]:
def recommend(matrix, id, num=5):
    index=df.index[df['id']==id].tolist()[0]
    similar_indexes = matrix[index].argsort()[:-(num+2):-1]
    print("Рекомендуем " + str(num) + " товаров, похожих на " + item(id))
    print("-------")
    for ind in similar_indexes[1:]:
        print("Товар " + item(df['id'][ind]) + " (score:" + str(matrix[index][ind]) + ")")

Осуществим рекомендацию

In [39]:
recommend(cosine_similarities_tfidf, id=24)

Рекомендуем 5 товаров, похожих на Cap 3 zip neck
-------
Товар Cap 3 crew (score:0.6843679481573043)
Товар Cap 3 crew (score:0.6309802636575079)
Товар Cap 3 bottoms (score:0.49913943613911466)
Товар Cap 3 bottoms (score:0.47940572685844857)
Товар Cap 2 zip neck (score:0.3089651968588024)


In [40]:
recommend(cosine_similarities_tfidf, id=42, num=3)

Рекомендуем 3 товаров, похожих на Freewheeler
-------
Товар Freewheeler max (score:0.6896990390184953)
Товар Mlc wheelie (score:0.21279952106418543)
Товар Mlc (score:0.1732850563925083)


# Word2Vec

Загрузим предобученную модель 

In [42]:
glove_vectors = gensim.downloader.load('glove-wiki-gigaword-300')

[==================================================] 100.0% 376.1/376.1MB downloaded


Напишем функцию для токенизации текста, которая возвращает список слов из текста

In [43]:
def tokenize_text(text):
    tokenizer = RegexpTokenizer(r"[\w-]+")
    html_pattern = re.compile('<.*?>')
    clean_text = tokenizer.tokenize(html_pattern.sub('', text.lower()))
    stops = set(stopwords.words("english"))
    sentence = [w for w in clean_text if not w in stops]
    return sentence

Напишем функцию для заполнения матрицы векторов по корпусу текстов, используя в качестве вектора текста среднее значение word2vec отдельных слов

In [44]:
def create_embed_matrix(texts):
    embed_matrix=np.zeros((len(texts),glove_vectors.vector_size))
    for ind, text in enumerate(texts):
        mean_word2vec=np.zeros(glove_vectors.vector_size)
        num_words=0
        tokenized_text=tokenize_text(text)
        for word in tokenized_text:
            if word in glove_vectors.index_to_key:
                mean_word2vec+=glove_vectors[word]
                num_words+=1
        embed_matrix[ind]=mean_word2vec/num_words
        embed_matrix[ind]=embed_matrix[ind]/np.sqrt(np.sum(embed_matrix[ind]**2)) 
    return embed_matrix

Заполняем матрицу векторов и считаем попарное косинусное расстояние

In [45]:
mean_word2vec_matrix=create_embed_matrix(df['description'].values)
cosine_similarities_word2vec = cosine_similarity(mean_word2vec_matrix, mean_word2vec_matrix)

Осуществим рекомендацию

In [47]:
recommend(cosine_similarities_word2vec, id=24)

Рекомендуем 5 товаров, похожих на Cap 3 zip neck
-------
Товар Cap 3 crew (score:0.9773523998282125)
Товар Cap 3 crew (score:0.9761014320846668)
Товар Cap 3 bottoms (score:0.9529411841100499)
Товар Cap 3 bottoms (score:0.9494847732576688)
Товар Cap 4 zip neck (score:0.9490495682783506)


In [48]:
recommend(cosine_similarities_word2vec, id=42, num=3)

Рекомендуем 3 товаров, похожих на Freewheeler
-------
Товар Freewheeler max (score:0.9855760794875849)
Товар Great divider (score:0.9483343742756004)
Товар Half mass (score:0.9450291297423384)
